In [1]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.pytorch import PyTorch
from sagemaker.tensorflow import TensorFlow
import os
from io import BytesIO

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"
role_name = role.split('/')[-1]  # Extract just the role name from the ARN

# Attach AdministratorAccess policy to your existing role
iam_client = boto3.client('iam')
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn="arn:aws:iam::aws:policy/AdministratorAccess"
)
print(f"Attached AdministratorAccess policy to role: {role}")

# Assume blood.csv is in S3 already - if not, upload it first
input_data_s3_uri = "s3://blue-blood-data/synthetic_data.csv"

bucket_name = "blue-blood-data"
region = 'us-east-1'
file_key = 'synthetic_data.csv'

/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/14/25 18:08:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=727928;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=326204;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ubaid/Library/Application Support/sagemaker/config.yaml


[04/14/25 18:08:17] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=261741;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=949434;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

[04/14/25 18:08:18] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=298957;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=103317;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

Attached AdministratorAccess policy to role: arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928


In [2]:
# First, make sure train.py exists in the notebook directory
if not os.path.exists("train.py"):
    print("Error: train.py not found in the current directory")
else:
    print("Found train.py in the current directory")
    
    # Upload train.py to S3 to ensure latest version is used
    code_prefix = "code"
    s3_code_path = sagemaker_session.upload_data("train.py", bucket=bucket_name, key_prefix=code_prefix)
    print(f"Uploaded train.py to {s3_code_path}")

# Verify that the S3 data file exists
try:
    s3_client = boto3.client('s3', region_name=region)
    s3_client.head_object(Bucket=bucket_name, Key=file_key)
    print(f"Verified that s3://{bucket_name}/{file_key} exists")
except Exception as e:
    print(f"Warning: Error verifying S3 file: {e}")

Found train.py in the current directory
Uploaded train.py to s3://blue-blood-data/code/train.py
Verified that s3://blue-blood-data/synthetic_data.csv exists


In [3]:
# Define hyperparameters for testing
# epochs_list = [50, 100, 150]
# lstm_units_list = [64, 128, 256]
# learning_rates = [0.001, 0.01, 0.1] 
# dropout_rates = [0.2, 0.3, 0.4]

epochs_list = [10]
lstm_units_list = [64]
learning_rates = [0.001] 
dropout_rates = [0.2]

print(f"Starting hyperparameter testing")

# test all combinations of hyperparameters
for epochs in epochs_list:
    for learning_rate in learning_rates:
        for lstm_units in lstm_units_list:
            for dropout_rate in dropout_rates:
                # Create a unique job name based on parameters
                job_name = f"lstm-e{epochs}-lr{learning_rate}-u{lstm_units}-d{int(dropout_rate*10)}-b1"
                job_name = job_name.replace('.', 'd')  # Replace dots with 'd' for valid job name
                print(f"\nStarting job: {job_name}")

                # Create the TensorFlow estimator with this set of hyperparameters
                estimator = TensorFlow(
                    entry_point='train.py',
                    role=role,
                    instance_count=1,
                    instance_type='ml.m5.4xlarge',
                    framework_version='2.9',
                    py_version='py39',
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': epochs,
                        'learning_rate': learning_rate,
                        'lstm_units': lstm_units,
                        'dropout_rate': dropout_rate,
                        'job_name': job_name
                    },
                )

                # Start training job
                estimator.fit({'train': input_data_s3_uri})



Starting hyperparameter testing

Starting job: lstm-e10-lr0d001-u64-d2-b1


[04/14/25 18:08:19] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=325305;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=890356;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=332560;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=247472;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[04/14/25 18:08:20] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=142921;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=921642;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[04/14/25 18:08:21] INFO     Creating training-job with name:                                       ]8;id=313798;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=648013;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tensorflow-training-2025-04-14-23-08-19-465                                           

2025-04-14 23:08:23 Starting - Starting the training job...
2025-04-14 23:08:38 Starting - Preparing the instances for training...
2025-04-14 23:09:13 Downloading - Downloading the training image......
2025-04-14 23:10:09 Training - Training image download completed. Training in progress...2025-04-14 23:10:38.771291: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-04-14 23:10:38.771460: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2025-04-14 23:10:38.797139: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2025-04-14 23:10:40,718 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2025-04-14 23:10:40,728 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-04-14